# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 김규범
- 학번: 20210516

여기 홍길동, 99999999 대신 본인의 이름, 학번 작성

-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [10]:
// Step 1: F와 S가 Comparable을 구현하도록 제약
record Pair<F extends Comparable<F>, S extends Comparable<S>>(F first, S second) implements Comparable<Pair<F, S>> {

    // Step 2: 제너릭을 사용하여 정적 팩토리 메소드 수정
    public static <F extends Comparable<F>, S extends Comparable<S>> Pair<F, S> of(F first, S second) {
        return new Pair<>(first, second);
    }

    // Step 3: compareTo 메소드 구현
    @Override
    public int compareTo(Pair<F, S> other) {
        // 먼저 'first' 요소를 비교
        int firstComparison = this.first.compareTo(other.first);
        if (firstComparison != 0) {
            return firstComparison;
        }
        // 'first'가 같다면 'second' 요소를 비교
        return this.second.compareTo(other.second);
    }
}


In [11]:
public class Main {
    public static void main(String[] args) {
        Pair<Integer, String> pair1 = Pair.of(1, "a");
        Pair<Integer, String> pair2 = Pair.of(2, "b");
        Pair<Integer, String> pair3 = Pair.of(1, "a");

        // 왼쪽이 더 작은 경우
        System.out.println(pair1.compareTo(pair2));  // 음수 값이 나와야 함
        // 왼쪽이 더 큰 경우
        System.out.println(pair2.compareTo(pair1));  // 양수 값이 나와야 함
        // 같은 경우
        System.out.println(pair1.compareTo(pair3));  // 0이 나와야 함
    }
}

------
## Part B: 가변 순서쌍 ModifiablePair

불변 순서쌍만을 priviate 필드로 갖고 있는 가변 순서쌍 제너릭 클래스를 작성하라

가변 순서쌍은 불변 순서쌍과 마찬가지 이름의 getter 메소드가 있고 추가로 setter 메소드도 있다.

또, 가변 순서쌍도 불변 순서쌍과 마찬가지로 Comparable 제너릭 인터페이스를 구현하라.


In [12]:
// Step 1: ModifiablePair 클래스 정의
class ModifiablePair<F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> {
    // 불변 순서쌍인 Pair를 private 필드로 선언
    private Pair<F, S> pair;

    // Step 2: 생성자에서 Pair 객체를 초기화
    ModifiablePair(F first, S second) {
        this.pair = Pair.of(first, second);
    }

    // Step 3: getter 메소드 (pair에게 위임)
    public F first() {
        return pair.first();
    }

    public S second() {
        return pair.second();
    }

    // Step 4: setter 메소드 (pair 객체를 새로 생성)
    public void setFirst(F first) {
        this.pair = Pair.of(first, pair.second());
    }

    public void setSecond(S second) {
        this.pair = Pair.of(pair.first(), second);
    }

    // Step 5: compareTo 메소드 (pair 객체에게 위임)
    @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }

    // toString 메소드 (pair에게 위임)
    @Override
    public String toString() {
        return pair.toString();
    }
}

In [13]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다.

In [8]:
public class Main {
    public static void main(String[] args) {
        // Step 1: ModifiablePair 객체 생성
        ModifiablePair<Integer, String> modPair1 = new ModifiablePair<>(1, "apple");
        ModifiablePair<Integer, String> modPair2 = new ModifiablePair<>(2, "banana");

        // Step 2: getter 메소드 테스트
        System.out.println("First of modPair1: " + modPair1.first());   // 출력: 1
        System.out.println("Second of modPair1: " + modPair1.second()); // 출력: apple

        // Step 3: setter 메소드 테스트
        modPair1.setFirst(3);
        modPair1.setSecond("orange");
        System.out.println("After modification: " + modPair1); // 출력: Pair(3, orange)

        // Step 4: compareTo 메소드 테스트
        // 왼쪽이 더 작은 경우
        System.out.println(modPair1.compareTo(modPair2)); // 양수 값 (3 > 2)
        // 왼쪽이 더 큰 경우
        modPair1.setFirst(1);
        System.out.println(modPair1.compareTo(modPair2)); // 음수 값 (1 < 2)
        // 같은 경우
        modPair1.setFirst(2);
        modPair1.setSecond("banana");
        System.out.println(modPair1.compareTo(modPair2)); // 0 (같음)
    }
}